In [1]:
import sys
import pickle
import numpy as np
import pynbody
import meshoid
from meshoid import Meshoid
from helpers.SimulationAnalysis import readHlist
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.colors as colors
# plt.rc("text", usetex=True)
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config InlineBackend.rc = {'figure.facecolor': 'w'}
sns.set_style("ticks")

ModuleNotFoundError: No module named 'pynbody'

In [3]:
def subselect_particles(f,host,distance_cut=1.5,projection_thickness=2.,Mpc_to_kpc=1000.):
    """
    Return particles within a radius of {distance_cut} kpc and a z-distance of +/-{projection_thickness} Mpc/h 
    from the host halo center
    
    Args:
        f (dictionary of pynbody objects): particle data at snapshot(s)
        host_halo (Rockstar object): host halo information
        distance_cut (float): radial distance from center of host to subselect, in kpc
        projection_thickness (float): absolute z-distance to subselect, in Mpc/h
        
    Returns:
        f_short (pynbody object): subselected particle data at snapshot(s)
    """
    
    xdist = f['pos'][:,0]-host['x']
    ydist = f['pos'][:,1]-host['y']
    zdist = f['pos'][:,2]-host['z']
    dist = Mpc_to_kpc*np.sqrt(xdist**2+ydist**2+zdist**2)/host['rvir']
    f_short = f[(dist<distance_cut)]# & (np.abs(zdist)<projection_thickness)]
        
    return f_short

In [4]:
with open("Halo004/sim_data.bin", "rb") as f:
    sim_data = pickle.load(f, encoding='latin1')
    
with open("Halo004/sim_data_16K.bin", "rb") as f:
    sim_data = pickle.load(f, encoding='latin1')
    
with open("Halo004/sim_data_running.bin", "rb") as f:
    sim_data_running = pickle.load(f, encoding='latin1')

FileNotFoundError: [Errno 2] No such file or directory: 'Halo004/sim_data.bin'

In [ ]:
custom_cmap = mpl.colors.LinearSegmentedColormap.from_list("", ["cyan","blue","#FF00FF","red","orange","yellow","white"])

In [ ]:
#custom_cmap = mpl.colors.LinearSegmentedColormap.from_list("", ["#607474", "#152539","black","#098585","#10fcfc","cyan","white"])

In [ ]:
def velocity_dispersion(velocities):
    mean_velocity = np.mean(velocities, axis=0)
    velocity_diff = velocities - mean_velocity
    velocity_dispersion = np.sqrt(np.sum(velocity_diff**2, axis=1))
    return velocity_dispersion

In [ ]:
brightness_density = norm(M.SurfaceDensity(0.1,0.9))
    
    
#trying to normalize my density data to the range [0,1] to then use here

#then after normalizing density, do some sort of 0.9 - new normalized value to get 

    
    
    
# don't think the above code is correct, but in theory I want to create a variable
# brightness_density that uses the density, so the lighter colors indicate
# higher density areas and the dark areas indicate low density

In [ ]:
BASE_PATH = ''

key = 'Halo004'
model = 'idm_1e-4GeV'
method = 'halfmode'

try:
    f = pynbody.load(BASE_PATH+'{}/{}_{}_16K/output_{}_{}/snapshot_235'.format(key,model,method,model,method))
    halos = readHlist(BASE_PATH+'{}/{}_{}_16K/output_{}_{}/hlist_1.00000.list'.format(key,model,method,model,method))
except:
    f = pynbody.load(BASE_PATH+'{}_16K/{}/output/snapshot_235'.format(key,model))
    halos = readHlist(BASE_PATH+'{}_16K/{}/output/hlist_1.00000.list'.format(key,model))
host = sim_data[key][model][method][0][0]
f_short = subselect_particles(f,host)

rho = f_short["rho"][:]
density_cut = (rho > .1)
pdata = {}
pdata['Masses'] = f_short['mass'][density_cut]
pdata['Coordinates'] = f_short['pos'][density_cut]
pdata['SmoothingLength'] = np.ones(len(f_short))[density_cut]*20/1000.
pdata['Velocities'] = f_short['vel'][density_cut]

pos = pdata["Coordinates"]*1000.
center = np.median(pos,axis=0)
center[0], center[1], center[2] = host['x']*1000, host['y']*1000, host['z']*1000
pos -= center
radius_cut = np.sum(pos*pos,axis=1) < 0.75*host['rvir']*0.75*host['rvir']
pos_new = pos
pos_new[:,1] = pos[:,2]
pos_new[:,2] = pos[:,1]
pos, mass, hsml, v = pos_new[radius_cut], pdata["Masses"][radius_cut], pdata["SmoothingLength"][radius_cut], pdata["Velocities"][radius_cut]

M = Meshoid(pos, mass, hsml)
rmax = 1.5*host['rvir']

res = 1000
X = Y = np.linspace(-rmax, rmax, res)
X, Y = np.meshgrid(X, Y)
sigma_gas_msun_pc2 = M.SurfaceDensity(M.m,center=np.array([0.,0.,0.]),size=1.5*host['rvir'],res=res)*1e4
sigma_gas_msun_pc2[np.sqrt(X**2 + Y**2) < 1.5*host['rvir']] = sigma_gas_msun_pc2[np.sqrt(X**2 + Y**2) < 1.5*host['rvir']].clip(min=0.8)

fig, ax = plt.subplots(figsize=(15,15))
p = ax.pcolormesh(X, Y, sigma_gas_msun_pc2.T, norm=colors.LogNorm(vmin=0.8,vmax=1e3),cmap=custom_cmap)            

ax.set_aspect('equal')
ax.set_xlabel("X (kpc)")
ax.set_ylabel("Y (kpc)")
ax.set_title(r'{}_{}'.format(key,model),fontsize=30)

ax.axis('off')
plt.show()

In [ ]:
# calculate the velocity dispersion
vdisp = np.std(v, axis=1)

# create the velocity dispersion map
vdisp_mesh = M.SurfaceDensity(vdisp, center=np.array([0.,0.,0.]), size=1.5*host['rvir'], res=res)

fig, ax = plt.subplots(figsize=(15,15))
p = ax.pcolormesh(X, Y, vdisp_mesh.T, alpha=brightness_density, cmap=custom_cmap, norm=colors.LogNorm(vmin=1e3,vmax=5e6)) 
ax.set_aspect('equal')
ax.set_xlabel("X (kpc)")
ax.set_ylabel("Y (kpc)")
ax.set_title(r'{}_{}'.format(key,model),fontsize=30)
ax.axis('off')

plt.savefig('velocity_dispersion.png')

plt.show()